## Radio interferometría y síntesis de imágenes en astronomía - Laboratorio 2

### Vicente Mieres

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# imports
import numpy as np
# modules
from modules.simulation import visibilities_simulation

In [3]:
# simulate visibilities no grid
VLA_L_4 = {
  "latitude": 34.078749,
  "longitude": -107.617728,
  "file_route": "../antenna_arrays/alma.cycle10.8.cfg",
  "catalog_source": "Sirius",
  "utc_start": "2024-10-21T00:00:00",
  "utc_end": "2024-10-21T06:00:00",
  "step_min": 5,
  "n_freqs": 4,
  "interferometer": {
    "name": "VLA",
    "band_name": "L" },
  "n_sources": 100,
  "max_offset_deg": 1.0,
  "flux_range": [0.1, 2.0],
  "seed": 42
  }

V, uvw_lambda, frequencies, baselines_enu = visibilities_simulation(VLA_L_4)

In [4]:
print(V.shape)

(903, 73, 4)


In [5]:
from modules.coords import max_basline
from modules.interferometry import grid_visibilities
import cupy as cp

# Resolucion
N = 4096
# Distancion maxima entre baselines
Dmax = max_basline(baselines_enu)
oversampling_factor = 10

c = 299792458.0 
freq = np.min(frequencies)
min_wavelenghgt = c / freq
dx = dy = (min_wavelenghgt / Dmax) / oversampling_factor

imgs = []
dus = []

du = 1.0 / (N * dx)
dv = du

In [6]:
import cupy as cp
print("CuPy CUDA runtime version:", cp.cuda.runtime.runtimeGetVersion())


CuPy CUDA runtime version: 12090


In [19]:
%%time
# CPU
VG2, WG2 = grid_visibilities(V, uvw_lambda, du, dv, Npix=N, use_gpu=False)

Gridding in CPU (NumPy Vectorized)
CPU times: total: 62.5 ms
Wall time: 38.8 ms


In [20]:
%%time
# GPU
VG, WG  = grid_visibilities(V, uvw_lambda, du, dv, Npix=N)

Gridding in GPU (CuPy Vectorized)
CPU times: total: 15.6 ms
Wall time: 14.5 ms


In [13]:
# Numba version of grid_visibilities (fully vectorized, no explicit loops)
from numba import njit
import numpy as np

@njit
def _grid_visibilities_numba_vectorized(u_all, v_all, V_all_real, V_all_imag, omega_all, du, dv, Npix):
    """
    Numba-optimized gridding function using fully vectorized operations.
    Uses linearized indices and bincount for accumulation - no explicit loops.
    """
    center = Npix // 2
    
    # Vectorized computation of grid indices
    i = np.rint(u_all / du).astype(np.int32) + center
    j = np.rint(v_all / dv).astype(np.int32) + center
    
    # Vectorized mask for valid indices
    mask = (i >= 0) & (i < Npix) & (j >= 0) & (j < Npix)
    
    # Apply mask
    i_valid = i[mask]
    j_valid = j[mask]
    V_real_valid = V_all_real[mask]
    V_imag_valid = V_all_imag[mask]
    omega_valid = omega_all[mask]
    
    # Compute weighted values (vectorized)
    values_real = omega_valid * V_real_valid
    values_imag = omega_valid * V_imag_valid
    
    # Linearize 2D indices to 1D for bincount
    linear_indices = j_valid * Npix + i_valid
    
    # Use bincount for accumulation (fully vectorized, no loops)
    # bincount automatically handles duplicate indices by summing
    VG_real_flat = np.bincount(linear_indices, weights=values_real, minlength=Npix*Npix)
    VG_imag_flat = np.bincount(linear_indices, weights=values_imag, minlength=Npix*Npix)
    WG_flat = np.bincount(linear_indices, weights=omega_valid, minlength=Npix*Npix)
    
    # Vectorized normalization on flattened arrays (Numba doesn't support 2D boolean indexing)
    valid_cells_flat = WG_flat > 0
    VG_real_flat[valid_cells_flat] /= WG_flat[valid_cells_flat]
    VG_imag_flat[valid_cells_flat] /= WG_flat[valid_cells_flat]
    
    # Reshape back to 2D
    VG_real = VG_real_flat.reshape((Npix, Npix)).astype(np.float32)
    VG_imag = VG_imag_flat.reshape((Npix, Npix)).astype(np.float32)
    WG = WG_flat.reshape((Npix, Npix)).astype(np.float32)
    
    # Combine real and imaginary parts (vectorized)
    VG = (VG_real + 1j * VG_imag).astype(np.complex64)
    
    return VG, WG


def grid_visibilities_numba(V, uvw_lambda, du, dv, Npix=256):
    """
    Grids complex visibilities onto a single (u, v) grid using Numba JIT compilation.
    Uses vectorized operations to minimize explicit loops.
    
    Parameters
    ----------
    V : array-like
        Complex visibilities, shape (n_baselines, n_times, n_freqs)
    uvw_lambda : array-like
        UVW coordinates in wavelengths, shape (n_baselines, n_times, n_freqs, 3)
    du, dv : float
        Grid spacing in u and v directions
    Npix : int
        Size of the output grid (Npix x Npix)
    
    Returns
    -------
    VG : np.ndarray
        Gridded visibilities, shape (Npix, Npix), dtype complex64
    WG : np.ndarray
        Grid weights, shape (Npix, Npix), dtype float32
    """
    print("Gridding in CPU (Numba JIT - Vectorized)")
    
    # Extract u and v coordinates
    u_coords = uvw_lambda[..., 0]
    v_coords = uvw_lambda[..., 1]
    
    # Flatten arrays
    u_all = u_coords.ravel()
    v_all = v_coords.ravel()
    V_all = V.ravel()
    omega_all = np.ones(len(V_all), dtype=np.float32)  # Pesos = 1
    
    # Convert to contiguous arrays with correct dtypes for Numba
    V_all_real = np.ascontiguousarray(V_all.real, dtype=np.float32)
    V_all_imag = np.ascontiguousarray(V_all.imag, dtype=np.float32)
    u_all_cont = np.ascontiguousarray(u_all, dtype=np.float32)
    v_all_cont = np.ascontiguousarray(v_all, dtype=np.float32)
    omega_all_cont = np.ascontiguousarray(omega_all, dtype=np.float32)
    
    # Call Numba-optimized core function
    VG, WG = _grid_visibilities_numba_vectorized(
        u_all_cont, v_all_cont, V_all_real, V_all_imag, 
        omega_all_cont, du, dv, Npix
    )
    
    return VG, WG


In [21]:
%%time
# Test Numba version
VG_numba, WG_numba = grid_visibilities_numba(V, uvw_lambda, du, dv, Npix=N)


Gridding in CPU (Numba JIT - Vectorized)
CPU times: total: 188 ms
Wall time: 187 ms


COMPARISON OF GRIDDED VISIBILITIES (VG)

Grid shape: (4096, 4096)
Data type: complex64

----------------------------------------------------------------------

GPU (CuPy) vs NumPy (Reference):
  Mean Absolute Error (MAE):        1.216748e-10
  Maximum Absolute Error:            2.861023e-06
  Root Mean Square Error (RMSE):    8.892375e-09
  Mean Relative Error:               5.556930e-09 (0.0000%)
  Maximum Relative Error:            1.405232e-06 (0.0001%)
  Mean Phase Difference:            1.444170e-11 rad (0.0000 deg)
  Maximum Phase Difference:         7.152557e-07 rad (0.0000 deg)
  Correlation Coefficient:           1.0000000000+0.0000000000j
  Arrays are close (rtol=1e-5, atol=1e-8): True

----------------------------------------------------------------------

Numba vs NumPy (Reference):
  Mean Absolute Error (MAE):        4.174641e-10
  Maximum Absolute Error:            2.132481e-06
  Root Mean Square Error (RMSE):    1.609182e-08
  Mean Relative Error:               1.667483e

(4096, 4096)